In [34]:
import torch
import torch.nn as nn
import numpy as np

from pathlib import Path

In [35]:
BS = 1
MELS = 50
DEVICE = 'cuda'
CHARSET = " .,!abcdefghijklmnopqrstuvwxyz"

In [36]:
# Model (Simple to get started)
"""
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.ln1 = nn.Sequential(
                                nn.Linear(50,100),
                                nn.ReLU(),                                
                                nn.Linear(100,250),
                                nn.ReLU(),
                                nn.Linear(250,500),
                                nn.ReLU(),
                                nn.Linear(500,500),
                                nn.ReLU(),
                                )

        self.gru = nn.GRU(500, 500, batch_first=False)
        self.ln2 = nn.Sequential(
                        nn.Linear(500, 250),
                        nn.ReLU(),
                        nn.Linear(250, 75),
                        nn.ReLU(),
                        nn.Linear(75,len(CHARSET)),
                         )

    def forward(self, x):
        x = self.ln1(x)
        x = nn.functional.relu(self.gru(x)[0])
        x = self.ln2(x)
        x = torch.nn.functional.log_softmax(x, dim=2)
        return x

model = Net().to(DEVICE)
"""


'\nclass Net(nn.Module):\n\n    def __init__(self):\n        super(Net, self).__init__()\n\n        self.ln1 = nn.Sequential(\n                                nn.Linear(50,100),\n                                nn.ReLU(),                                \n                                nn.Linear(100,250),\n                                nn.ReLU(),\n                                nn.Linear(250,500),\n                                nn.ReLU(),\n                                nn.Linear(500,500),\n                                nn.ReLU(),\n                                )\n\n        self.gru = nn.GRU(500, 500, batch_first=False)\n        self.ln2 = nn.Sequential(\n                        nn.Linear(500, 250),\n                        nn.ReLU(),\n                        nn.Linear(250, 75),\n                        nn.ReLU(),\n                        nn.Linear(75,len(CHARSET)),\n                         )\n\n    def forward(self, x):\n        x = self.ln1(x)\n        x = nn.functional.re

In [37]:
# Model

# TODO: CLEAN UP THIS!

# NOTE: Do nut use below 64 ATM due to last linear layer
C = 16 # BUG: Can we risk clipping for n value of C?
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv = nn.Sequential(
                                nn.Conv2d(1,C, kernel_size=3, stride=1, padding='same'),
                                nn.ReLU(),
                                nn.BatchNorm2d(C),
                                nn.Conv2d(C,C*2, kernel_size=3, stride=1, padding='same'),
                                nn.ReLU(),
                                nn.BatchNorm2d(C*2),
                                )                          
                                

        self.ln1 = nn.Sequential(
                                nn.Linear((C*2)*50,600),
                                nn.ReLU(),                                
                                nn.Linear(600,300),
                                nn.ReLU(),
                                )

        self.gru = nn.GRU(300, 300, batch_first=False)
        self.ln2 = nn.Sequential(
                        nn.Linear(300, 150),
                        nn.ReLU(),
                        nn.Linear(150, 75),
                        nn.ReLU(),
                        nn.Linear(75,len(CHARSET)),
                         )

    def forward(self, x):
        # TODO: Use different implementation/Input when using CONV?
        # TODO: When using conv should we change input ? Input ATM: (Time,Batch,Mel/Freq) 
        # TODO: CLEAN UP and make sur it is correct 
        x = x.reshape(x.shape[1],1,-1,MELS)
        #print(x.shape)
        x = self.conv(x)
        #print(x.shape)
        x = x.permute(0, 2, 1, 3)
        #print("dis" + str(x.shape))
        x = x.reshape(x.shape[0],-1,(C*2)*MELS)
        x = x.permute(1,0,2)
        #print(x.shape)
        x = self.ln1(x)
        #print(x.shape)
        x = nn.functional.relu(self.gru(x)[0])
        #print(x.shape)
        x = self.ln2(x)
        #print(x.shape)
        #  Output (Time,Batch,Mel/Freq) -> Convention when using CTC loss 
        x = torch.nn.functional.log_softmax(x, dim=2)
        return x

model = Net().to(DEVICE)
model


Net(
  (conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ln1): Sequential(
    (0): Linear(in_features=1600, out_features=600, bias=True)
    (1): ReLU()
    (2): Linear(in_features=600, out_features=300, bias=True)
    (3): ReLU()
  )
  (gru): GRU(300, 300)
  (ln2): Sequential(
    (0): Linear(in_features=300, out_features=150, bias=True)
    (1): ReLU()
    (2): Linear(in_features=150, out_features=75, bias=True)
    (3): ReLU()
    (4): Linear(in_features=75, out_features=30, bias=True)
  )
)

In [76]:
# TEST
MODEL = 200
TEST = 3
TEST2 = [1,2,3,14,15]

model.load_state_dict(torch.load(f"models/{str(MODEL)}-c.pt"))

FOLDER = Path('data/LJSpeech-1.1/wavs')

path_numpy = FOLDER.parent / Path('numpy')

path_data = path_numpy / Path('data')
path_labels = path_numpy / Path('labels')

for index, i in enumerate(range(len(TEST2))):

    data = np.load(path_data / Path(str(TEST) + '.npy'))
    data = torch.tensor(data[index]).float()
    #data = torch.nn.functional.normalize(data)
    data = data[:,:,None].permute(1,2,0).to(DEVICE)
    # print(data.shape)

    # Run the model
    output = model(data)
    output = output[:, 0, :].argmax(dim=1)
    #print(output[0:800])
    estimate = []
    for char in output:
        if char != 0:
            estimate.append(CHARSET[char-1])
    print("GUESS-> " + ''.join(estimate))

    # VERIFY
    labels = np.load(path_labels / Path(str(TEST) + '.npy'))

    correct = []
    for char in labels[index]:
        if char != 0:
            correct.append(CHARSET[char-1])
    print("Label-> " + ''.join(correct))

GUESS-> e e e ee e e ee o e e e e e e e e e e e e e e e e e e e e  e 
Label-> a little reduced in ugliness. the design of the letters of this modern  old style  leaves a good deal to be desired,
GUESS->  e e e e e  e ee  e e e e e e e e e
Label-> and the whole effect is a little too gray, owing to the thinness of the letters.
GUESS-> e e e e e e e e e e   e e e e e e ee e e e e e e e e e e e e e e  e  ee
Label-> it must be remembered, however, that most modern printing is done by machinery on soft paper, and not by the hand press,
GUESS-> ee e e e ee e e e e e e e e e ee e e e  e e e e e e e e e e e
Label-> and these somewhat wiry letters are suitable for the machine process, which would not do justice to letters of more generous design.
GUESS->  a e   e e e  e e e e e e e e e e e e ee e eeoe.
Label-> it is discouraging to note that the improvement of the last fifty years is almost wholly confined to great britain.
